# Исследование надёжности заёмщиков

## Описание проекта
Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Инструкция по выполнению
### Шаг 1. Откройте таблицу и изучите общую информацию о данных
Путь к файлу: data.csv. Скачать датасет
### Шаг 2. Предобработка данных

    определите и заполните пропущенные значения:
        опишите, какие пропущенные значения вы обнаружили;
        приведите возможные причины появления пропусков в данных;
        объясните, по какому принципу заполнены пропуски;
    замените вещественный тип данных на целочисленный:
        поясните, как выбирали метод для изменения типа данных;
    удалите дубликаты:
        поясните, как выбирали метод для поиска и удаления дубликатов в данных;
        приведите возможные причины появления дубликатов;
    выделите леммы в значениях столбца с целями получения кредита:
        опишите, как вы проводили лемматизацию целей кредита;
    категоризируйте данные:
        перечислите, какие «словари» вы выделили для этого набора данных, и объясните, почему.

В данных могут встречаться артефакты — значения, которые не отражают действительность. Например, отрицательное количество дней трудового стажа. Для реальных данных — это нормально. Нужно описать возможные причины появления таких данных и обработать их.
### Шаг 3. Ответьте на вопросы

    Есть ли зависимость между наличием детей и возвратом кредита в срок?
    Есть ли зависимость между семейным положением и возвратом кредита в срок?
    Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
    Как разные цели кредита влияют на его возврат в срок?

Ответы сопроводите интерпретацией — поясните, о чём именно говорит полученный вами результат.
### Шаг 4. Напишите общий вывод
Оформление: Задание выполните в Jupyter Notebook. Программный код заполните в ячейках типа code, текстовые пояснения — в ячейках типа markdown. Примените форматирование и заголовки.
Описание данных

    children — количество детей в семье
    days_employed — общий трудовой стаж в днях
    dob_years — возраст клиента в годах
    education — уровень образования клиента
    education_id — идентификатор уровня образования
    family_status — семейное положение
    family_status_id — идентификатор семейного положения
    gender — пол клиента
    income_type — тип занятости
    debt — имел ли задолженность по возврату кредитов
    total_income — ежемесячный доход
    purpose — цель получения кредита

### Шаг 1. Откройте таблицу и изучите общую информацию о данных¶

In [1]:
import pandas as pd
df = pd.read_csv('data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [2]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


#### Вывод
Всего в таблице 12 столбцов.
Проблемы:
1. Пропущены значения в days_employed, total_income.
2. Количество детей -1 в столбце children (47 строк). Количество детей 20 в столбце children (76 строк): удалим их, так как они занимают 0.2% всех данных.
3. Отрицательные значения в столбце days_employed, пропущены значения.
4. Возраст клиента в столбце dob_years имеет значение 0. Заменим средним значением, так как от медианы оно сильно не отличается.
5. Уровень образования в столбце education имеет верхний регистр. Приведем к нижнему регистру.
6. Семейное положение в столбце family_status приведем к нижнему регистру.
7. Гендер в столце gender имеет значение XNA (1 строка). Удалим ее без потери данных.

### Шаг 2. Предобработка данных
#### 1. Обработка пропусков

In [3]:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В days_employed и total_income пропущено 2174 значений. Выясним, зависят ли они друг от друга.

In [4]:
df[(df['days_employed'].isna() == True) & (df['total_income'].isna() == True)].shape

(2174, 12)

Выяснили, что в столбцах days_employed и total_income пропуски не случайны и зависят друг от друга. Данные пропуски не будем удалять, так как они составляют 10% от всех данных.
В столбце days_employed (стаж работы в днях) существуют достаточно крупные значения, которые, скорее всего, выражены в часах. Максимальное значение 401755. Если поделить 401755 на 365 дней в году, а после на 24 часа в сутки, то получим 45 лет стажа работы. Допустим, что это максимальный стаж работы. Тогда все значения, которые выше 45 лет, будет делить на 24 часа, чтобы привести их к дням.
С помощью метода abs() изменим отрицательные значения на положительные.

In [5]:
df['days_employed'] = df['days_employed'].abs()

In [6]:
def convert_hours_to_days(hours):
    if hours > 45 * 365:
        return hours / 24
    else:
        return hours
df['days_employed'] = df['days_employed'].apply(convert_hours_to_days)

In [7]:
median_days = df['days_employed'].median()

In [8]:
df['days_employed'] = df['days_employed'].fillna(median_days)

In [9]:
df['days_employed'].isna().sum()

0

Мы избавились от отрицательных значений с помощью метода abs, далее с помощью метода apply применили функцию, которая уменьшает значения и приводит их к дням. После заменили все пропущенные значения медианным. Убедились, что пропущенные значения отсутствуют.

Пропуски в столбце total_income заменим медианными значениями по категориям занятости (income_type). Создадим сводную таблицу, переведем ее в словарь. Создадим функцию, которую применим в столбцу income_type с помощью apply. На вход функция будет принимать строку датафрейма, а возвращать медианное значение по ключу словаря.
Применим с помощью apply функцию к тем ячейкам, где есть пропуски в total_income.

In [10]:
medians = df.groupby('income_type')['total_income'].median()

In [11]:
medians

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64

In [12]:
df.loc[df['total_income'].isna(), 'total_income']

12      NaN
26      NaN
29      NaN
41      NaN
55      NaN
         ..
21489   NaN
21495   NaN
21497   NaN
21502   NaN
21510   NaN
Name: total_income, Length: 2174, dtype: float64

In [13]:
def fillna_median(row):
    return medians[row]
df.loc[df['total_income'].isna(), 'total_income'] = df['income_type'].apply(fillna_median)

In [14]:
df['total_income'].isna().sum()

0

Пропущенные значения заполнили медианными значениями по группе занятости.

В столбце children существуют значения -1 и 20. Удалим их, так как объем данных занимает 0.2% от всего датасета. Возраст в столбце dob_years заменим на среднее значение. Пол XNA в столбце gender удалим, так это всего лишь единственная строка.

In [15]:
df = df[(df['children'] != -1) & (df['children'] != 20)]

In [16]:
dob_years_mean = int(df['dob_years'].mean())
df['dob_years'] = df['dob_years'].replace(0, dob_years_mean)
# df['children'].unique()
# df['dob_years'].unique()

Убедились, что некорректные значения и пропуски отсутсвуют.

#### Вывод
Пропущенные значения в столбцах days_employed и total_income пропуски не случайны и зависят друг от друга. Возможно, намеренно пропущены значения, так как клиенты боялись, что это повлияет на получение кредита. Заполнили данные значения медианой, так как среднее из-за выбросов искажает данные.

Удалили строки с некорректными значениями в столбце children -1 и 20, так как это не влияет на объем данных.

В столбце dob_years возраста заменили 0 на среднее значение. Возможно, это ошибка при заполнении анкеты при заявке на получение кредита.

#### 2. Изменение типов данных
Стаж и доход заменим на тип int, так как он будет проще восприниматься. Использует метод astype.

In [17]:
df['days_employed'] = df['days_employed'].astype(int)
df['total_income'] = df['total_income'].astype(int)

Заменили тип данных float на тип данных int для читабельности датасета с помощью astype(int).

#### 3. Поиск дубликатов
Проверим, какое количество дубликатов. Далее заменим верхний регистр в столбцах education и family_status на нижний регистр. И заново посчитаем количество дубликатов. Их оказалось 71. Удалим данные дубликаты, заменяя данные.

In [18]:
df.duplicated().sum()

54

In [19]:
df['education'] = df['education'].str.lower()
df['family_status'] = df['family_status'].str.lower()

In [20]:
df.duplicated().sum()

71

In [21]:
df = df.drop_duplicates().reset_index(drop = True)

In [22]:
len(df)

21331

#### Вывод
Удалены 71 грубый дубликат, заменены индексы на новые с помощью метода reset_index(drop=True). Причина дубликатов - верхний регистр.

#### 4. Лемматизация
В столбце purpose сделаем лемматизацию, чтобы группировать цели кредита.

In [27]:
from pymystem3 import Mystem
m = Mystem()

Installing mystem to /home/yana/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


In [28]:
df['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [29]:
lemma = df['purpose'].apply(m.lemmatize)

In [30]:
lemma

0                             [покупка,  , жилье, \n]
1                   [приобретение,  , автомобиль, \n]
2                             [покупка,  , жилье, \n]
3                [дополнительный,  , образование, \n]
4                           [сыграть,  , свадьба, \n]
                             ...                     
21326                  [операция,  , с,  , жилье, \n]
21327               [сделка,  , с,  , автомобиль, \n]
21328                              [недвижимость, \n]
21329    [на,  , покупка,  , свой,  , автомобиль, \n]
21330             [на,  , покупка,  , автомобиль, \n]
Name: purpose, Length: 21331, dtype: object

In [31]:
lemma = lemma.str.join('')

In [32]:
lemma

0                     покупка жилье\n
1           приобретение автомобиль\n
2                     покупка жилье\n
3        дополнительный образование\n
4                   сыграть свадьба\n
                     ...             
21326              операция с жилье\n
21327           сделка с автомобиль\n
21328                  недвижимость\n
21329    на покупка свой автомобиль\n
21330         на покупка автомобиль\n
Name: purpose, Length: 21331, dtype: object

In [33]:
from collections import Counter

In [34]:
Counter(lemma)

Counter({'покупка жилье\n': 640,
         'приобретение автомобиль\n': 459,
         'дополнительный образование\n': 455,
         'сыграть свадьба\n': 760,
         'операция с жилье\n': 647,
         'образование\n': 442,
         'на проведение свадьба\n': 763,
         'покупка жилье для семья\n': 637,
         'покупка недвижимость\n': 616,
         'покупка коммерческий недвижимость\n': 658,
         'покупка жилой недвижимость\n': 602,
         'строительство собственный недвижимость\n': 628,
         'недвижимость\n': 631,
         'строительство недвижимость\n': 619,
         'на покупка подержать автомобиль\n': 471,
         'на покупка свой автомобиль\n': 504,
         'операция с коммерческий недвижимость\n': 645,
         'строительство жилой недвижимость\n': 620,
         'жилье\n': 641,
         'операция со свой недвижимость\n': 623,
         'автомобиль\n': 967,
         'заниматься образование\n': 408,
         'сделка с подержанный автомобиль\n': 481,
         'получ

In [35]:
type(lemma)

pandas.core.series.Series

#### Вывод
Выделяем 4 цели кредита: недвижимость, автомобиль, свадьба, образование.

#### 4. Категоризация данных

In [38]:
def purpose_cat(list):
    if 'автомобиль' in list:
        return 'автомобиль'
    
    elif 'жилье' in list:
        return 'жилье'
    
    elif 'недвижимость' in list:
        return 'жилье'
    
    elif 'образование' in list:
        return 'образование'
    
    else:
        return 'свадьба'

In [39]:
df['purpose_category'] = lemma.apply(purpose_cat)
df['purpose_category'].value_counts()

жилье          10751
автомобиль      4279
образование     3988
свадьба         2313
Name: purpose_category, dtype: int64

#### Вывод
Большую часть занимает цель кредита - жилье. Далее идет цель - автомобиль, после образование и свадьба. Можно было бы разделить категорию "жилье" на подкатегории, но тогда бы получилось много категорий.

### Шаг 3. Ответьте на вопросы
1. Есть ли зависимость между наличием детей и возвратом кредита в срок?

Сгруппируем таблицу по столбцу children и к столбцу debt применим функции count и mean.

In [40]:
df.groupby('children').agg({'debt': ['count', 'mean']})

debt          
          count      mean
children                 
0         14091  0.075438
1          4808  0.092346
2          2052  0.094542
3           330  0.081818
4            41  0.097561
5             9  0.000000

#### Вывод
1. Да, зависимость от количества детей на возврат кредита в срок имеется. Наибольшая задолженность по кредиту у группы с 4 детьми. Меньше всего задолженность у тех, у кого нет детей. Группу с 5 детьми не рассматриваем, так как ее значения не значительны.

2. Есть ли зависимость между семейным положением и возвратом кредита в срок?

Сгруппируем таблицу по столбцу family_status и к столбцу debt применим функции count и mean.

In [41]:
df.groupby('family_status').agg({'debt': ['count', 'mean']})

debt          
                       count      mean
family_status                         
в разводе               1189  0.070648
вдовец / вдова           951  0.066246
гражданский брак        4134  0.093130
женат / замужем        12261  0.075606
не женат / не замужем   2796  0.097639

#### Вывод
Наибольшая задолженность у тех, кто холост. Наименьшая задолженность у категории вдовцов , женатых и в разводе, так как супруги могли эффективнее выплачивать кредит.
Связь между семейный положением и возвратом кредита в срок так же присутсвует.

3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Рассмотрим подробнее столбец total_income, чтобы в дальнейшем разделить заемщиков на категории дохода.

In [44]:
df['total_income'].describe().round()

count      21331.0
mean      165343.0
std        98314.0
min        20667.0
25%       107507.0
50%       142594.0
75%       195842.0
max      2265604.0
Name: total_income, dtype: float64

Разделим доходы на 3 категории: низкий, средний, высокий. Низкий до 107507, средний от 107507 до 195842, высокий от 195842.

In [45]:
def total_income_groups(total_income):
    if total_income <= 107507:
        return 'низкий'
    elif 107507 < total_income <= 195842:
        return 'средний'
    elif total_income > 195842:
        return 'высокий'

In [46]:
df['total_income_group'] = df['total_income'].apply(total_income_groups)

In [47]:
df.groupby('total_income_group').agg({'debt': ['count', 'mean']})

debt          
                    count      mean
total_income_group                 
высокий              5332  0.071455
низкий               5333  0.080068
средний             10666  0.086630

#### Вывод
Люди с высоким уровнем дохода имеют процент задолженности ниже, чем люди с низким и средним уровнем доходов. Большее количество людей оказались в средней группе.

4. Как разные цели кредита влияют на его возврат в срок?

In [48]:
df.groupby('purpose_category').agg({'debt': ['count', 'mean']})

debt          
                  count      mean
purpose_category                 
автомобиль         4279  0.093480
жилье             10751  0.072551
образование        3988  0.092528
свадьба            2313  0.079118

#### Вывод
Наибольший процент задолженности по кредиту у людей с целью покупки автомобиля и образования. Вероятно, что покупают автомобиль и образование молодые люди, которые еще не успевают устроиться на работу после окончания образования. Поэтому растут задолженности. Наименьший процент задолженности у тех, кто берет кредит на покупку жилья.

### Шаг 4. Общий вывод
Целью исследования является вопрос: влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Необходимо проанализировать данные столбцов children и family_status.

In [ ]:
df.groupby(['family_status', 'children']).agg({'debt': ['count', 'mean']})

#### Вывод
В целом, прослеживается связь между задолженностью и количеством детей, а также семейным положением. Чем больше детей у людей, тем выше задолженность. Семейное положение влияет на возврат кредита в срок. Меньше всего задолженность у тех, кто вдовец или женат. Больше всего задолженность у тех, кто находится в гражданском браке и у тех, кто холост.